In [ ]:
from IPython.core.display import HTML
with open ("style.css", "r") as file:
    css = file.read()
HTML(css)

# Minimizing a <span style="font-variant:small-caps;">Fsm</span>

The function `arb(M)` takes a non-empty set `M` as its argument and returns an *arbitrary* element from this set.
The set `M` is not changed.

In [ ]:
def arb(M):
    for x in M:
        return x
    assert False, 'Error: arb called with empty set!'

The function `cart_prod(A, B)` computes the *Cartesian product* $A \times B$ of the sets $A$ and $B$ where $A \times B$ is defined as follows:
$$ A \times B := \{ (x, y) \mid x \in A \wedge y \in B \}. $$

In [ ]:
def cart_prod(A, B):
    return { (x, y) for x in A for y in B }

The function `separate` takes four arguments:
- `Pairs`  a set `Pairs` of pairs of states from some given <span style="font-variant:small-caps;">Fsm</span> $F$.

   If $(p_1, p_2) \in \texttt{Pairs}$, then $p_1$ and $p_2$ are known to be *separable*.
- `States` is the set of all states of the <span style="font-variant:small-caps;">Fsm</span> $F$,
- `Σ` is the alphabet of the <span style="font-variant:small-caps;">Fsm</span> $F$.
- `𝛿` is the transition function of the <span style="font-variant:small-caps;">Fsm</span>

The function `separate(Pairs, States, Σ, 𝛿)` computes the set of pairs of states $(q_1, q_2)$ that are separable because there is some character $c \in \Sigma$ such that 
$$\delta(q_1,c) = p_1, \quad \textrm{but} \quad \delta(q_2,c) = p_2. $$

In [ ]:
def separate(Pairs, States, Σ, 𝛿):
    Result = { (q1, q2) for q1 in States
                        for q2 in States
                        for c  in Σ 
                        if (𝛿[q1, c], 𝛿[q2, c]) in Pairs
             }
    return Result

Given a state `p` and a `Partition` of the set of all states, the function `find_equivalence_class(p, Partition)` returns the equivalence class of `p`, i.e. it returns the set from `Partition` that contains `x`. 

In [ ]:
def find_equivalence_class(p, Partition):
    return arb({ C for C in Partition if p in C })

The function `reachable(q0, Σ, 𝛿)` takes three arguments:
* `q0` is the start state of an Fsm,
* `Σ`  is the alphabet.
* `𝛿`  is the transition function.  The transition function is assumed to be *complete*. `𝛿` is represented as a dictionary.   

It returns the set of all states that can be reached from the start state `q0` by reading strings of characters from `Σ`.

In [ ]:
def reachable(q0, Σ, 𝛿):
    Result = { q0 }
    while True:
        NewStates = { 𝛿[p, c] for p in Result for c in Σ }
        if NewStates <= Result:
            return Result
        Result |= NewStates

The function `all_separable(Q, A, Σ, 𝛿)` takes four arguments:
* `Q` is the set of states of the Fsm.
* `A` is the set of all accepting states,
* `Σ`  is the alphabet.
* `𝛿` is the transition function.  

  `𝛿` is represented as a dictionary.   

The function computes the set of all Pairs `(p, q)` such that `p` and `q` are separable, i.e. all pairs such that
$$ \exists s \in \Sigma^*: \bigl(\delta^*(p, s) \in A \wedge \delta^*(q,s) \not\in A\bigr) \vee 
                           \bigl(\delta^*(p, s) \not\in A \wedge \delta^*(q,s) \in A\bigr). 
$$

In [ ]:
def all_separable(Q, A, Σ, 𝛿):
        Separable = cart_prod(Q - A, A) | cart_prod(A, Q - A)
        while True:
            NewPairs = separate(Separable, Q, Σ, 𝛿)
            if NewPairs <= Separable:
                return Separable
            Separable |= NewPairs

The function `minimize(A)` takes a deterministic 
<span style="font-variant:small-caps;">Fsm</span> `F` as its input.
Here `F` is a 5-tuple of the form
$$ F = (Q, \Sigma, \delta, q_0, A) $$
The algorithm performs the following steps:
1. All unreachable states are eliminated.
2. All accepting states are separated form all non-accepting states.
3. States are separated as long as possible.
   Two states $p_1$ and $p_2$ are separable if there is a character 
   $c \in \Sigma$ such that 
   $$\delta(p_1,c) = q_1, \quad \delta(p_2,c) = q_2, \quad \textrm{and} \quad
     \mbox{$q_1$ and $q_2$ are separable.}
   $$
4. States that are not separable are *equivalent* and are therefore identified and grouped
   in equivalence classes.  The states in an equivalence class are then identified.

In [ ]:
def minimize(F):
    Q, Σ, 𝛿, q0, A = F
    Q            = reachable(q0, Σ, 𝛿)
    Separable    = all_separable(Q, A, Σ, 𝛿)
    Equivalent   = cart_prod(Q, Q) - Separable
    EquivClasses = { frozenset({ p for p in Q if (p, q) in Equivalent })
                     for q in Q
                   }
    newQ0        = arb({ M for M in EquivClasses if q0 in M })
    newAccept    = { M for M in EquivClasses if arb(M) in A }   
    newDelta     = {}
    for q in Q:
        for c in Σ:
            p = 𝛿.get((q, c))
            if p != None:
                classOfP = find_equivalence_class(p, EquivClasses)
                classOfQ = find_equivalence_class(q, EquivClasses)
                newDelta[(classOfQ, c)] = classOfP
            else:
                newDelta[(classOfQ, c)] = frozenset()
    return EquivClasses, Σ, newDelta, newQ0, newAccept